# 🧩 월마트 실업률 예측 모델링 (회귀 과제)
- 빅데이터 분석기사 실기 연습을 위해 제가 만든 자료입니다.
- 데이터셋은 kaggle의 데이터셋을 제가 다시 가공한 뒤 X_train, y_train, X_test로 분리하였습니다. 또한 파생 변수를 추출하여 컬럼으로 만들었습니다. 원본 데이터셋은 다음의 주소에서 확인하실 수 있습니다 : https://www.kaggle.com/datasets/yasserh/walmart-dataset

### 🧩 문제
- 주어진 학습용 데이터 x_train.csv를 활용하여 실업률(Unemployment)를 예측하는 모형을 만들고, 평가용 데이터 x_test.csv에 적용하여 실업률을 예측하여 csv 파일로 생성하시오.(평가 지표는 rmse)

### 🧩 힌트
- 평가지표 rmse로 보아 회귀 과제임을 알 수 있습니다.
- 회귀과제에서는 predict_proba가 아닌 predict가 사용됩니다. predict_proba는 어떠한 클래스에 속할 확률값을 구하는 데 주로 사용되기 때문에 분류 과제에서 사용됩니다. 하지만 회귀 과제는 연속형 변수를 예측해야 하기 때문에 predict를 사용합니다.

# 1. 필요한 패키지 불러오기
- 기본 패키지를 불러옵니다.

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# 2. 데이터 파일 읽어오기

In [2]:
x_train = pd.read_csv('./Walmart_x_train.csv')
x_test = pd.read_csv('./Walmart_x_test.csv')
y_train = pd.read_csv('./Walmart_y_train.csv')

In [3]:
x_train.head(1)

,wal_id,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Year,Month
0,3855,27,1557485.75,0,66.1,4.203,142.679817,2012,9


In [4]:
x_test.head(1)

,wal_id,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Year,Month
0,122,1,1697230.96,0,78.3,3.452,221.749484,2012,6


In [5]:
y_train.head(1)

,wal_id,Unemployment
0,3855,8.239


- 💦 시험에서는 x_test 고유의 id를 사용하여 예측값을 파일로 제출하라는 형식을 요구할 것입니다. 그 형식에 맞게 테스트셋의 고유 id인 wal_id를 미리 복사하여 저장합니다.

In [6]:
wal_id = x_test['wal_id'].copy()

- wal_id 컬럼은 모델 학습에는 필요하지 않으므로 drop() 처리합니다. 

In [7]:
x_train = x_train.drop(columns = 'wal_id')
x_test =  x_test.drop(columns = 'wal_id')
y_train = y_train.drop(columns = 'wal_id')

In [8]:
x_train.head(1)

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Year,Month
0,27,1557485.75,0,66.1,4.203,142.679817,2012,9


In [9]:
y_train.head(1)

,Unemployment
0,8.239


# 3. info() 함수로 기본 정보 확인하기
- 행과 열의 수, 결측치, 데이터 유형을 확인합니다.

In [10]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         5148 non-null   int64  
 1   Weekly_Sales  5147 non-null   float64
 2   Holiday_Flag  5148 non-null   int64  
 3   Temperature   5148 non-null   float64
 4   Fuel_Price    5148 non-null   float64
 5   CPI           5147 non-null   float64
 6   Year          5148 non-null   int64  
 7   Month         5148 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 321.9 KB


In [11]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         1287 non-null   int64  
 1   Weekly_Sales  1287 non-null   float64
 2   Holiday_Flag  1287 non-null   int64  
 3   Temperature   1287 non-null   float64
 4   Fuel_Price    1287 non-null   float64
 5   CPI           1287 non-null   float64
 6   Year          1287 non-null   int64  
 7   Month         1287 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 80.6 KB


In [12]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unemployment  5147 non-null   float64
dtypes: float64(1)
memory usage: 40.3 KB


# 4. 결측치 처리
- x_train의 Weekly_Sales 컬럼과 CPI 컬럼에 각각 1개의 결측치가 존재하고, y_train의 Unemployment 컬럼에 1개의 결측치가 존재합니다.
- 여기서는 dropna()를 사용하여 결측치가 존재하는 모든 행을 제거하려고 합니다.
- 💦 x_train의 인덱스와 y_train의 인덱스가 달라지지 않도록 concat한 뒤, 다시 x_train과 y_train으로 나눕니다.
- 두 개의 데이터프레임을 concat할 때에는 axis = 1을 반드시 입력합니다.

In [13]:
train = pd.concat([x_train, y_train], axis = 1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5148 entries, 0 to 5147
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         5148 non-null   int64  
 1   Weekly_Sales  5147 non-null   float64
 2   Holiday_Flag  5148 non-null   int64  
 3   Temperature   5148 non-null   float64
 4   Fuel_Price    5148 non-null   float64
 5   CPI           5147 non-null   float64
 6   Year          5148 non-null   int64  
 7   Month         5148 non-null   int64  
 8   Unemployment  5147 non-null   float64
dtypes: float64(5), int64(4)
memory usage: 362.1 KB


In [14]:
train = train.dropna()
train

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Year,Month,Unemployment
0,27,1557485.75,0,66.10,4.203,142.679817,2012,9,8.239
1,32,1185391.96,0,42.76,3.550,195.331898,2011,10,8.513
2,6,1524390.07,0,63.89,3.308,219.789714,2011,11,6.551
3,32,1129422.86,1,23.34,3.103,196.919506,2012,2,8.256
6,26,889670.29,0,36.90,4.025,137.868000,2012,4,7.489
...,...,...,...,...,...,...,...,...,...
5143,41,1560590.05,0,65.77,3.545,198.100106,2012,8,6.432
5144,43,557543.62,0,44.56,2.708,203.571429,2010,12,10.210
5145,20,1910177.38,0,74.57,2.784,204.729325,2010,8,7.527
5146,40,943237.12,0,64.05,3.820,136.183129,2011,8,4.584


In [15]:
x_train = train[['Store', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Year', 'Month']]
y_train = train[['Unemployment']]

# 5. describe() 함수로 요약 통계량 확인
- 이상치가 있는지 확인합니다.

In [16]:
x_train.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Year,Month
count,5146.000000,5.146000e+03,5146.000000,5146.000000,5146.000000,5146.000000,5146.000000,5146.000000
mean,23.005441,1.041722e+06,0.072095,60.694267,3.351968,171.544773,2010.960358,6.445783
std,13.040058,5.647160e+05,0.258670,18.487510,0.458683,39.400615,0.798676,3.244353
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,2010.000000,1.000000
25%,12.000000,5.456197e+05,0.000000,47.510000,2.921500,131.586613,2010.000000,4.000000
50%,23.000000,9.571723e+05,0.000000,62.960000,3.433000,182.619515,2011.000000,6.000000
75%,34.000000,1.419281e+06,0.000000,74.945000,3.730000,212.728547,2012.000000,9.000000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.214288,2012.000000,12.000000


In [17]:
x_test.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Year,Month
count,1287.000000,1.287000e+03,1287.000000,1287.000000,1287.000000,1287.000000,1287.000000,1287.000000
mean,22.977467,1.068809e+06,0.061383,60.528928,3.384134,171.709918,2010.982906,6.458430
std,12.773062,5.626198e+05,0.240125,18.291357,0.459153,39.190642,0.790507,3.216347
min,1.000000,2.200604e+05,0.000000,7.460000,2.514000,126.064000,2010.000000,1.000000
25%,12.000000,5.838246e+05,0.000000,47.265000,2.952000,132.485467,2010.000000,4.000000
50%,23.000000,9.724054e+05,0.000000,61.700000,3.488000,182.598306,2011.000000,6.000000
75%,34.000000,1.422766e+06,0.000000,74.950000,3.763500,212.847386,2012.000000,9.000000
max,45.000000,3.766687e+06,1.000000,99.660000,4.468000,227.232807,2012.000000,12.000000


In [18]:
y_train.describe()

,Unemployment
count,5146.000000
mean,8.011432
std,1.886987
min,3.879000
25%,6.895000
50%,7.874000
75%,8.622000
max,14.313000


# 6. 학습용 / 검증용 데이터셋 분리
- 💦 회귀 과제에서는 train_test_split 함수에 stratify를 사용하지 않습니다. 각 클래스의 비율이 모델 학습에 영향을 미칠 수 있는 분류 과제와 달리, 회귀 과제는 연속형 값을 예측하는 것이므로 클래스의 비율은 고려할 필요가 없습니다.

In [19]:
from sklearn.model_selection import train_test_split

x_tr, x_valid, y_tr, y_valid = train_test_split(x_train, y_train['Unemployment'], 
                                                test_size = 0.3)

# 7. 모델 학습
- 분류 과제와 회귀 과제에서 모두 사용할 수 있는 랜덤 포레스트 모델을 사용합니다.
- 회귀 과제에서는 RandomForestRegressor를 임포트하여 사용하고, 분류 과제에서는 RandomForestClassifier를 임포트하여 사용합니다.

In [20]:
from sklearn.ensemble import RandomForestRegressor

modelRF = RandomForestRegressor()
modelRF.fit(x_train, y_train)

RandomForestRegressor()

# 8. 모델 평가
- 회귀 과제이므로 predict_proba가 아닌 predict를 사용합니다.
- 평가 지표로는 rmse를 사용합니다. rmse는 예측 값과 실제 값 간의 평균적인 차이를 나타냅니다.
- 함께 평가 지표로 r2 score(결정 계수)가 사용됩니다. 이는 모델이 종속 변수의 변동성을 얼마나 잘 설명하는지를 나타내며, 값이 1에 가까울수록 모델의 설명력이 높다고 할 수 있습니다.

In [21]:
y_validation = modelRF.predict(x_valid)

In [22]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_valid, y_validation)
rmse = mean_squared_error(y_valid, y_validation, squared = False)
r2score = r2_score(y_valid, y_validation)
print(mse)
print(rmse)
print(r2score)

0.008496854821308296
0.09217838586842522
0.9975620375508546


In [23]:
pred = modelRF.predict(x_test)

In [24]:
pred

array([7.09038, 6.4939 , 9.00086, ..., 6.759  , 7.88976, 8.94594])

# 9. 파일 제출

In [25]:
result = pd.DataFrame({'wal_id':wal_id, 'Unemployment':pred}).to_csv('walmart_submission.csv', 
                                                                       index = False)

In [26]:
result = pd.read_csv('./walmart_submission.csv')
result

,wal_id,Unemployment
0,122,7.09038
1,640,6.49390
2,4455,9.00086
3,4406,6.89100
4,276,6.61459
...,...,...
1282,1566,6.34324
1283,6071,10.56753
1284,5816,6.75900
1285,3449,7.88976
